# Import

In [9]:
import pandas as pd
from pathlib import Path
import sqlite3 as sqlite
import numpy as np

# Connect to database

In [10]:
database_path = Path("detox/DeTox-Dataset_complete.sqlite3") # path to your database file
dbconnect = None
cursor = None

if not database_path.is_file():
    print(f"Database {database_path} does not exist. Creating a new database now ...")
try:
        # open database connection
        dbconnect = sqlite.connect(database_path, detect_types=sqlite.PARSE_DECLTYPES | sqlite.PARSE_COLNAMES)
        cursor = dbconnect.cursor()
        # Check Foreign-Key Constraints can be switched off if needed with the following line:
        # cursor.execute("PRAGMA foreign_keys = OFF;")
except sqlite.Error as e:
        # if errors occur
        print("Error %s:" % e.args[0])

## Modify how date is saved

In [11]:
# change date to valid format
cursor.execute('UPDATE Comments SET "date" = substr("date", 1, instr("date", "+")-1) WHERE date LIKE "%+%"')
dbconnect.commit()

# Modify data

In [12]:
data = pd.read_sql_query('SELECT c_id, c_text, "date", conv_id from Comments NATURAL JOIN Goldstandard ORDER BY conv_id desc, "date";', con=dbconnect)
data.head(100)

,c_id,c_text,date,conv_id
0,1406777427234373632,@reitschuster Man sollte keine Diplom-Volkswir...,2021-06-21 00:54:09,1406756979880742912
1,1406751155707842561,"Das ist, mit Verlaub, Quatsch. Ct-Werte wurden...",2021-06-20 23:09:46,1406751155707842561
2,1406751925983395851,@Karl_Lauterbach Aber nach Impfung gibt der Ct...,2021-06-20 23:12:49,1406751155707842561
3,1406755191706243077,@travelflo @Karl_Lauterbach und trotzdem ander...,2021-06-20 23:25:48,1406751155707842561
4,1406755713548959745,"@DeineMeinungung @Karl_Lauterbach nochmal, die...",2021-06-20 23:27:53,1406751155707842561
...,...,...,...,...
95,1406748783791255552,"@travelflo @Karl_Lauterbach Meister, es forder...",2021-06-20 23:00:20,1406745651220713479
96,1406753568699437063,@Karl_Lauterbach Das ist ziemlicher Blödsinn. ...,2021-06-20 23:19:21,1406745651220713479
97,1406715695828770825,@reitschuster Nicht mehr Impfschäden als bei a...,2021-06-20 20:48:52,1406711193490931715
98,1406710447894630406,@Karl_Lauterbach Wäre gut wenn wir erstmal uns...,2021-06-20 20:28:00,1406710025951850500


In [13]:
# Important: close the database connection at the end
dbconnect.close();

In [14]:
reaction1 = np.full(len(data), None)
reaction2 = np.full(len(data), None)

for index, row in data.iterrows():
    if (row['conv_id'] is not None):

        if  index + 1 < len(data) and data.iloc[index + 1]["conv_id"] == row['conv_id']:
            reaction1[index] = data.iloc[index + 1]["c_text"]
            
        if  index + 2 < len(data) and data.iloc[index + 2]["conv_id"] == row['conv_id']:
            reaction2[index] = data.iloc[index + 2]["c_text"]
    #data[['reaction1', 'reaction2']] = data.apply(get_reactions, axis=1) 
    
data['reaction1'] = reaction1
data['reaction2'] = reaction2

data.head(100)

,c_id,c_text,date,conv_id,reaction1,reaction2
0,1406777427234373632,@reitschuster Man sollte keine Diplom-Volkswir...,2021-06-21 00:54:09,1406756979880742912,None,None
1,1406751155707842561,"Das ist, mit Verlaub, Quatsch. Ct-Werte wurden...",2021-06-20 23:09:46,1406751155707842561,@Karl_Lauterbach Aber nach Impfung gibt der Ct...,@travelflo @Karl_Lauterbach und trotzdem ander...
2,1406751925983395851,@Karl_Lauterbach Aber nach Impfung gibt der Ct...,2021-06-20 23:12:49,1406751155707842561,@travelflo @Karl_Lauterbach und trotzdem ander...,"@DeineMeinungung @Karl_Lauterbach nochmal, die..."
3,1406755191706243077,@travelflo @Karl_Lauterbach und trotzdem ander...,2021-06-20 23:25:48,1406751155707842561,"@DeineMeinungung @Karl_Lauterbach nochmal, die...",@Karl_Lauterbach Was für einen Schwachsinn sie...
4,1406755713548959745,"@DeineMeinungung @Karl_Lauterbach nochmal, die...",2021-06-20 23:27:53,1406751155707842561,@Karl_Lauterbach Was für einen Schwachsinn sie...,"@Karl_Lauterbach Da ist BS, ct-Wert wurde nich..."
...,...,...,...,...,...,...
95,1406748783791255552,"@travelflo @Karl_Lauterbach Meister, es forder...",2021-06-20 23:00:20,1406745651220713479,@Karl_Lauterbach Das ist ziemlicher Blödsinn. ...,None
96,1406753568699437063,@Karl_Lauterbach Das ist ziemlicher Blödsinn. ...,2021-06-20 23:19:21,1406745651220713479,None,None
97,1406715695828770825,@reitschuster Nicht mehr Impfschäden als bei a...,2021-06-20 20:48:52,1406711193490931715,None,None
98,1406710447894630406,@Karl_Lauterbach Wäre gut wenn wir erstmal uns...,2021-06-20 20:28:00,1406710025951850500,None,None


In [15]:
# Randomize data
data = data.sample(frac=1)

# Write to CSV

In [16]:
data.to_csv('01_comments_with_immediate_conversations-random.csv', sep=';', index=False, header=True) 